# 基于Transformers的命名实体识别

## 1.导包

In [1]:
from transformers import AutoTokenizer,AutoModelForTokenClassification,TrainingArguments,Trainer,DataCollatorForTokenClassification
import evaluate
from datasets import load_dataset
import torch
from torch.utils.data import Dataset,DataLoader

## 2.加载数据集

In [2]:
ner_dataset = load_dataset("lansinuote/peoples-daily-ner")

In [6]:
ner_dataset["train"]

Dataset({
    features: ['id', 'tokens', 'ner_tags'],
    num_rows: 20865
})

In [7]:
ner_dataset["train"].features

{'id': Value('string'),
 'tokens': List(Value('string')),
 'ner_tags': List(ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']))}

In [11]:
ner_dataset["train"].features["ner_tags"]

List(ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']))

In [12]:
ner_dataset["train"].features["ner_tags"].feature

ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'])